# Decorators

## Overview - Day 1

**Decorators abstract functionality from a function so it can be applied to multiple functions**
- The concept is similar to that of `pytest` fixtures.
    - If you have to repeat the same code in each `pytest` test, define a `fixture` instead, and pass the `fixture` to the `pytest` function.
- If you have to define the same behavior in multiple functions, use a **decorator** instead.
    - An example of this might be a function that checks to see if a login session is active, with every request.
    - Why write that code into every function when you can just decorate the functions and accomplish the same thing?

A decorator can add behavior to a function:
- Functions are passed into a decorator.
- The decorator can do something to the function before or after the function runs.
- Returns the newly decorated object.

---

### Decorator Example

#### Import the required modules

In [1]:
# `wraps` is not required, although it is preferred
from functools import wraps
import time

#### Create a basic template for defining a decorator

In [5]:
def my_first_decorator(function):
    @wraps(function)
    # *args and **kwargs are placeholders for arguments/keyword arguments from the function passed to the decorator
    def wrapper(*args, **kwargs):
        # Do something before running the function passed to the decorator
        # Call the function passed to the decorator
        result = function(*args, **kwargs)
        # Do something after running the function passed to the decorator
        return result
    # return `wrapper` which is the decorated function
    return wrapper

#### Use the decorator to wrap the function

The decorator syntax uses the @ symbol to prefix the decorator function.

In [3]:
@my_first_decorator
def my_function(args):
    pass

The decorator syntax is is syntactic sugar for:

In [8]:
# This is NOT the common way to use a decorator
def my_new_function(args):
    pass

my_new_function = my_first_decorator(my_new_function)

---

### `*args` and `**kwargs` parameters in decorators

Python function arguments include:
- **positional**
    - Mandatory and have no default values.
- **keyword**
    - Optional and have defaults.
- **arbitrary**
    - `*args` becomes a `tuple` of all remaining (not explicitly defined) **positional** arguments.
    - `*kwargs` becomes a `dict` of all remaining (not explicitly defined) **keyword** arguments.

>Note: The parameter names `*args` and `**kwargs` are arbitrary.
> - These could instead be `*seeds` and `**nuts`.

In [13]:
# Example
def test_args(first_name, *args, **kwargs):
    ''' The parameters *args and **kwargs are optional and capture any
        unspecified positional and keyword arguments, respectively.
    '''

    # Return a list of all arguments
    return_value = {
        'first_name': first_name,
        'args': args,
        'kwargs': kwargs
    }

    return return_value

''' When calling the function, any keyword arguments whether defined explicitly or not, must come after any positional arguments
'''
test_args('Tim', 'W', 'Hull', age=41, state='Oregon')

{'first_name': 'Tim',
 'args': ('W', 'Hull'),
 'kwargs': {'age': 41, 'state': 'Oregon'}}

#### Create a decorator to display arguments

- Display `*args` before calling the function passed to the decorator.
- Display `**kwargs` after calling the function passed to the decorator.

In [32]:
from functools import wraps

def display_args(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        print('\nThis decorator displays *args values before calling the function:'
              f'{args}\n')

        # Call the function passed to the decorator
        result = function(*args, **kwargs)

        print('And also displays **kwargs values after calling the function:'
              f'{kwargs}\n')

        return result

    # Return wrapper as a decorated function
    return wrapper

In [33]:
@display_args
def send_args(*args, **kwargs):
    print('\tThis is a decorated function\n')

send_args('Tim', 'Hull', hair='blonde', eyes='blue')


This decorator displays *args values before calling the function:('Tim', 'Hull')

	This is a decorated function

And also displays **kwargs values after calling the function:{'hair': 'blonde', 'eyes': 'blue'}



---

## Review - Day 2

### Create a new decorator and apply it to a new function

In [1]:
# Import modules
from functools import wraps
from time import ctime, time

In [2]:
# Create decorator function
def add_times(wrapped_function):
    @wraps(wrapped_function)

    # Define a wrapper function
    def wrapper(*args, **kwargs):
        # Display the epoch before the wrapped function
        print(f'\nThe current epoch time is {time()}')

        # Call the wrapped function
        result = wrapped_function(*args, **kwargs)

        # Display the time after the wrapped function
        print(f'The current time is {ctime()}\n')

        # return the result of the wrapepd function
        return result

    # Return the result of the wrapper function
    return wrapper

In [3]:
# Define a new function using the decorator
@add_times
def my_name(name):
    print(f'\tMy name is {name}')

In [4]:
# Call the wrapped function
my_name('Tim')


The current epoch time is 1628967894.996227
	My name is Tim
The current time is Sat Aug 14 19:04:54 2021



---

## Write a Time-It Decorator - Day 2

In [9]:
# Import Modules
from functools import wraps
from time import time
from typing import Union

In [16]:
# Create the decorator function
def time_it(function):
    ''' This function is a decorator which displays the total time it takes
        for the decorated function to run.

        Args:
            function (function):
                Decorated function.

        Returns:
            None.
    '''

    ''' Call the @wraps decorator on the decorated function.
        This preserves the docstring in the wrapped function.
    '''
    @wraps(function)
    def wrapper(*args, **kwargs):

        # Capture the time before the wrapped function starts
        start_time = time()

        # Call the decorated function
        result = function(*args, **kwargs)

        # Capture the time after the wrapped function ends
        end_time = time()
        total_time = round(start_time - end_time, 5)

        # Display the time the wrapped function tool to run
        print(f'The "{function.__name__}" function took {total_time} to run.')

        # return the result of the wrapped function
        return result

    # Return the result of the wrapper function
    return wrapper


In [17]:
# Create a decorated function to time
@time_it
def get_exponent(num: Union[int, float], exp: int = 1) -> Union[int, float]:
    ''' Determine the exponent of a number.

        Args:
            num (int or float):
                Base number.
            exp (int):
                Exponent to apply to `num`.

        Returns:
            exponent (int or float):
                Result of the exponent operation.
    '''

    # Determine the exponent
    exponent = num ** exp

    # Display the result
    print(f'The exponent of {num} to the {exp} power is {exponent}.')

    return exponent

In [24]:
# Call the decorated function
exp = get_exponent(10, 10)

The exponent of 10 to the 10 power is 10000000000.
The "get_exponent" function took -0.00017 to run.


### Test the use of the @wraps decorator

`@wraps` should ensure the docstring of the wrapped function is preserved

In [25]:
# Look up the docstring for the wrapped function
help(get_exponent)

Help on function get_exponent in module __main__:

get_exponent(num: Union[int, float], exp: int = 1) -> Union[int, float]
    Determine the exponent of a number.
    
    Args:
        num (int or float):
            Base number.
        exp (int):
            Exponent to apply to `num`.
    
    Returns:
        exponent (int or float):
            Result of the exponent operation.



### Redfine the decorator function without `@wraps`

In [26]:
# Create the decorator function
def time_it(function):
    ''' This function is a decorator which displays the total time it takes
        for the decorated function to run.

        Args:
            function (function):
                Decorated function.

        Returns:
            None.
    '''

    ''' Call the @wraps decorator on the decorated function.
        This preserves the docstring in the wrapped function.
    '''
    def wrapper(*args, **kwargs):

        # Capture the time before the wrapped function starts
        start_time = time()

        # Call the decorated function
        result = function(*args, **kwargs)

        # Capture the time after the wrapped function ends
        end_time = time()
        total_time = round(start_time - end_time, 5)

        # Display the time the wrapped function tool to run
        print(f'The "{function.__name__}" function took {total_time} to run.')

        # return the result of the wrapped function
        return result

    # Return the result of the wrapper function
    return wrapper


### Redefine the wrapped function

Notice that the absense of the `@wraps` decorator in the `time_it` (decorator) function prevents the dosctring from being available in the wrapped function (`get_exponent`)

In [28]:
# Create a decorated function to time
@time_it
def get_exponent(num: Union[int, float], exp: int = 1) -> Union[int, float]:
    ''' Determine the exponent of a number.

        Args:
            num (int or float):
                Base number.
            exp (int):
                Exponent to apply to `num`.

        Returns:
            exponent (int or float):
                Result of the exponent operation.
    '''

    # Determine the exponent
    exponent = num ** exp

    # Display the result
    print(f'The exponent of {num} to the {exp} power is {exponent}.')

    return exponent

help(get_exponent)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



## Stacking Decorators - Day 3

Decorators may be stacked which allows the application of multiple decorators to a single function.

```python
# Example
@decorator_1  # Does something before or after `decorator_2` and `my_function`
@decorator_2  # Does something before or after `my_function`
def my_function():
    pass
```

### Create a function with stacked decorators

- The purpose of the function `delay` is to use the `time.sleep` function to generate a random delay.
- The purpose of the first decorator function `display_details` is to wrap and display information about `delay`.
    - `__doc__`
    - `*args` and `**kwargs`
- The purpose of the second decorator function `timer` is to wrap both `delay` and `display_details`, providing a output including:
    - Start time
    - End time
    - Total run time

In [2]:
# Import modules
from functools import wraps
from itertools import cycle
from random import randint
from sys import stdout
from time import sleep, time

In [3]:
# `display_details` decorator function
def display_details(function):
    ''' Displays details for a wrapped function.

        Args:
            function (function):
                Decorated function.

        Returns:
            wrapper (function):
                Result of decorated function.
    '''

    @wraps(function)
    def wrapper(*args, **kwargs):

        # Display information about the decorated function
        # Docstring
        print(f'\n\tThe docstring for the function {function.__name__} is:\n'
              f'\t{function.__doc__}')

        # *args
        if len(args) > 0:
            print(f'\n\tThe positional arguments for the function {function.__name__} are:')
            for arg in args:
                print(f'\t-{arg}')
        else:
            print(f'\n\tNo positional arguments in {function.__name__}.')

        # **kwargs
        if len(kwargs) > 0:
            print(f'\n\tThe keyword arguments for the function {function.__name__} are:')
            for kwarg in kwargs:
                print(f'\t-{kwarg}')
        else:
            print(f'\n\tNo keyword arguments in {function.__name__}.')

        # Call the wrapped function
        function(*args, **kwargs)

    # Return the result of the wrapped function
    return wrapper

In [4]:
# `timer` decorator function
def timer(function):
    ''' Displays start, end, and total time for a wrapped function.

        Args:
            function (function):
                Decorated function.

        Returns:
            wrapper (function):
                Result of decorated function.
    '''

    @wraps(function)
    def wrapper(*args, **kwargs):

        # Capture and display the start time
        start = time()
        print(f'\nThe function start time is {start}.')

        # Call the wrapped function
        function(*args, **kwargs)

        # Capture and display the end time and total time
        end = time()
        total = end - start
        print(f'\nThe function end time is {start} '
              f'and the total run time is {total}.')

    # Return the result of the wrapped function
    return wrapper

In [5]:
# `delay` function with nested decorators
@timer  # Call this decorator first, to wrap/decorate all nested decorators and functions
@display_details  # Call this decorator second, to include its processing time in the `@timer` calculation
def delay(
    first_name: str,
    favorite_number: int,
    favorite_programming_language: str = 'Python',
    favorite_season: str = 'Summer',
    *args: any,
    **kwargs: any
) -> None:
    ''' Introduces a random delay.

        Args:
            first_name (str):
                Your first name.
            favorite_number (int):
                Your favorite number.
            favorite_programming_language (str):
                Your favorite programming language.
            favorite_season (str):
                Your favorite season of the year.

        Returns:
            None.
    '''

    print(f'\nHello, {first_name}:\n'
          f'\tYour favorite number is {favorite_number}.\n'
          f'\tYour favorite programming language is {favorite_programming_language}.\n'
          f'\tYour favorite season of the year is {favorite_season}.')

    wait = cycle(['.  ', '.. ', '...'])

    timer = 0
    delay = randint(5, 15)
    print(f"Let's wait for {delay / 2} seconds")
    while delay > timer:
        stdout.write(f'\r{next(wait)}')
        stdout.flush()
        timer += 1
        sleep(.5)

In [8]:
# Call the `delay` function
delay(
    'Tim',
    18,
    favorite_programming_language='Python',
    favorite_food='Pizza',
    favorite_season='Fall'
)


The function start time is 1629051850.167829.

	The docstring for the function delay is:
	 Introduces a random delay.

        Args:
            first_name (str):
                Your first name.
            favorite_number (int):
                Your favorite number.
            favorite_programming_language (str):
                Your favorite programming language.
            favorite_season (str):
                Your favorite season of the year.

        Returns:
            None.
    

	The positional arguments for the function delay are:
	-Tim
	-18

	The keyword arguments for the function delay are:
	-favorite_programming_language
	-favorite_food
	-favorite_season

Hello, Tim:
	Your favorite number is 18.
	Your favorite programming language is Python.
	Your favorite season of the year is Fall.
Let's wait for 5.5 seconds
.. 
The function end time is 1629051850.167829 and the total run time is 5.538745403289795.
